## Aulas de ETL



In [1]:
#pip install pandera

In [2]:
# importando bibliotecas
import pandas as pd
import pandera as pa
import warnings
import numpy as np

In [65]:
# importando o dataframe
df = pd.read_csv('ocorrencia_2010_2020.csv', sep=';',
                 parse_dates=['ocorrencia_dia'], dayfirst=True)


In [4]:
df.head()

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,***,***,RIO DE JANEIRO,RJ,...,2010-01-03,12:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,...,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,...,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,...,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO


In [66]:
# droppndo as colunas desnecessárias
df = df.drop(['ocorrencia_latitude', 'ocorrencia_longitude', 'codigo_ocorrencia1', 'codigo_ocorrencia3', 'codigo_ocorrencia4', 'ocorrencia_pais', 'investigacao_aeronave_liberada',
              'investigacao_status', 'divulgacao_relatorio_numero', 'divulgacao_relatorio_publicado', 'divulgacao_dia_publicacao',
              'total_aeronaves_envolvidas', 'total_aeronaves_envolvidas', 'ocorrencia_saida_pista'], axis=1)


df.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


## Validação de dados

In [67]:
schema = pa.DataFrameSchema(
    columns={

        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        'ocorrencia_classificacao': pa.Column(pa.String),
        'ocorrencia_cidade': pa.Column(pa.String),
        'ocorrencia_uf': pa.Column(pa.String, pa.Check.str_length(2, 2), nullable = True),
        'ocorrencia_aerodromo': pa.Column(pa.String, nullable =  True),
        'ocorrencia_dia': pa.Column(pa.DateTime),
        'ocorrencia_hora': pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        'total_recomendacoes': pa.Column(pa.Int)
    }
)
# checando se é uma expressão regular de tempo/formato: "pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$')"

In [68]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


In [69]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [9]:
# Extraindo dia e mês
df.ocorrencia_dia.dt.month

0        1
1        1
2        1
3        1
4        1
        ..
5747    12
5748    12
5749    12
5750    12
5751    12
Name: ocorrencia_dia, Length: 5752, dtype: int64

In [70]:
#procurando dados nulos
df.isnull().sum()

codigo_ocorrencia           0
codigo_ocorrencia2          0
ocorrencia_classificacao    0
ocorrencia_cidade           0
ocorrencia_uf               0
ocorrencia_aerodromo        0
ocorrencia_dia              0
ocorrencia_hora             1
total_recomendacoes         0
dtype: int64

## Limpeza de dados

In [11]:
#localizando dados
print(df.loc[1, 'ocorrencia_cidade'])
print(df.loc[3])


BELÉM
codigo_ocorrencia                         39527
codigo_ocorrencia2                        39527
ocorrencia_classificacao               ACIDENTE
ocorrencia_cidade            LUCAS DO RIO VERDE
ocorrencia_uf                                MT
ocorrencia_aerodromo                       ****
ocorrencia_dia              2010-01-04 00:00:00
ocorrencia_hora                        17:30:00
total_recomendacoes                           0
Name: 3, dtype: object


In [12]:
print(df.loc[1:3])

   codigo_ocorrencia  codigo_ocorrencia2 ocorrencia_classificacao  \
1              40349               40349                INCIDENTE   
2              40351               40351                INCIDENTE   
3              39527               39527                 ACIDENTE   

    ocorrencia_cidade ocorrencia_uf ocorrencia_aerodromo ocorrencia_dia  \
1               BELÉM            PA                 SBBE     2010-01-03   
2      RIO DE JANEIRO            RJ                 SBRJ     2010-01-03   
3  LUCAS DO RIO VERDE            MT                 ****     2010-01-04   

  ocorrencia_hora  total_recomendacoes  
1        11:05:00                    0  
2        03:00:00                    0  
3        17:30:00                    0  


In [37]:
# buscar duas linhas
df.loc[{10, 40}]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
40,39158,INCIDENTE,BELÉM,PA,<NA>,2010-01-28,16:00:00,0
10,39789,INCIDENTE,SÃO PEDRO DO SUL,RS,<NA>,2010-01-10,21:30:00,0


In [38]:
# Somente a coluna
df.loc[:, 'ocorrencia_cidade']

0           RIO DE JANEIRO
1                    BELÉM
2           RIO DE JANEIRO
3       LUCAS DO RIO VERDE
4                  PELOTAS
               ...        
5747              CAMPINAS
5748     LAGOA DA CONFUSÃO
5749        RIO DE JANEIRO
5750             VICENTINA
5751        RIO DE JANEIRO
Name: ocorrencia_cidade, Length: 5752, dtype: object

In [71]:
# verificando se o codigo de ocorrencia é de valor unico
df.codigo_ocorrencia.is_unique

True

In [72]:
# transformando o codigo_ocorrencia em indice
df.set_index('codigo_ocorrencia', inplace=True)

In [40]:
df.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


In [73]:
df.reset_index(drop = True, inplace = True)

In [19]:
df.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


# Alteração dos dados

In [74]:
# alterando o nome do aerodromo

df.loc[0, 'ocorrencia_aerodromo'] = pd.NA

In [41]:
df.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


### Limpeza de fato
 |Ocorrencia UF | Ocorrencia aerodromo | Ocorrencia Hora |
 | - | - | - |
 | ** | ###!|null|
 |   | #### |  |
 |  | **** |  |
 |  | ** * ****** | |
 
 

In [75]:
#Limpando a linha do aerodromo
df.loc[df.ocorrencia_aerodromo == '****', ['ocorrencia_aerodromo']] = pd.NA
df.loc[df.ocorrencia_aerodromo == '***', ['ocorrencia_aerodromo']] = pd.NA
df.loc[df.ocorrencia_aerodromo == '**', ['ocorrencia_aerodromo']] = pd.NA
df.loc[df.ocorrencia_aerodromo == 'NULL', ['ocorrencia_aerodromo']] = pd.NA
df.loc[df.ocorrencia_aerodromo == '###!', ['ocorrencia_aerodromo']] = pd.NA
df.loc[df.ocorrencia_aerodromo == '####', ['ocorrencia_aerodromo']] = pd.NA

In [23]:
#Limpando em massa usando o replace - problema no Pandas =s
#df.replace(['**','###!','####','****','*****','NULL'], pd.NA, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\missing.py:47: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\missing.py:58: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


In [76]:

df.isna().sum()

codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  0
ocorrencia_aerodromo        2199
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [81]:
df.fillna(np.nan)

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
5747,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00,0
5749,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,INCIDENTE GRAVE,VICENTINA,MS,NaN,2020-12-31,09:00:00,0


In [80]:
df.dropna(subset=['ocorrencia_uf'])

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
5747,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0
5749,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,INCIDENTE GRAVE,VICENTINA,MS,<NA>,2020-12-31,09:00:00,0


## Transformação dos dados

In [46]:
df.dtypes

codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [47]:
#localizar pelo índice
df.iloc[3]

codigo_ocorrencia2                        39527
ocorrencia_classificacao               ACIDENTE
ocorrencia_cidade            LUCAS DO RIO VERDE
ocorrencia_uf                                MT
ocorrencia_aerodromo                       <NA>
ocorrencia_dia              2010-01-04 00:00:00
ocorrencia_hora                        17:30:00
total_recomendacoes                           0
Name: 3, dtype: object

In [50]:
df.loc[-1]  #Nao funciona

KeyError: -1

In [49]:
df.iloc[-1]

codigo_ocorrencia2                        79844
ocorrencia_classificacao              INCIDENTE
ocorrencia_cidade                RIO DE JANEIRO
ocorrencia_uf                                RJ
ocorrencia_aerodromo                       SBJR
ocorrencia_dia              2020-12-31 00:00:00
ocorrencia_hora                        13:24:00
total_recomendacoes                           0
Name: 5751, dtype: object

In [51]:
df.iloc[10:15]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
10,39789,INCIDENTE,SÃO PEDRO DO SUL,RS,<NA>,2010-01-10,21:30:00,0
11,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
12,40414,INCIDENTE,BRASÍLIA,DF,SBBR,2010-01-10,03:00:00,0
13,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,<NA>,2010-01-11,17:00:00,2
14,40107,ACIDENTE,GURUPÁ,PA,SNGU,2010-01-12,14:30:00,0


In [52]:
df.loc[10:15]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
10,39789,INCIDENTE,SÃO PEDRO DO SUL,RS,<NA>,2010-01-10,21:30:00,0
11,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
12,40414,INCIDENTE,BRASÍLIA,DF,SBBR,2010-01-10,03:00:00,0
13,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,<NA>,2010-01-11,17:00:00,2
14,40107,ACIDENTE,GURUPÁ,PA,SNGU,2010-01-12,14:30:00,0
15,39315,ACIDENTE,CANUTAMA,AM,<NA>,2010-01-15,21:22:00,0


In [53]:
df.loc[:, 'ocorrencia_uf']

0       RJ
1       PA
2       RJ
3       MT
4       RS
        ..
5747    SP
5748    TO
5749    RJ
5750    MS
5751    RJ
Name: ocorrencia_uf, Length: 5752, dtype: object

In [54]:
df['ocorrencia_uf']

0       RJ
1       PA
2       RJ
3       MT
4       RS
        ..
5747    SP
5748    TO
5749    RJ
5750    MS
5751    RJ
Name: ocorrencia_uf, Length: 5752, dtype: object

In [84]:
# manipulação de valores nulos
df.isna().sum()
df.isnull().sum()

codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  0
ocorrencia_aerodromo        2199
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [63]:
df.ocorrencia_uf.isnull().sum()

0

In [83]:
filtro = df.ocorrencia_aerodromo.isnull()
df.loc[filtro]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
5,39807,INCIDENTE,SALVADOR,BA,<NA>,2010-01-06,17:53:00,0
7,39707,INCIDENTE GRAVE,CANUTAMA,AM,<NA>,2010-01-09,12:30:00,3
9,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,<NA>,2010-01-10,20:00:00,0
...,...,...,...,...,...,...,...,...
5741,79754,INCIDENTE GRAVE,NOVA MARINGÁ,MT,<NA>,2020-12-28,13:00:00,0
5744,79753,INCIDENTE GRAVE,GOIATUBA,GO,<NA>,2020-12-29,12:00:00,0
5745,79755,ACIDENTE,MATO RICO,PR,<NA>,2020-12-29,10:30:00,0
5748,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0


In [85]:
filtroHora = df.ocorrencia_hora.isnull()
df.loc[filtroHora]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
4100,78823,INCIDENTE,CORUMBÁ,MS,*****,2017-07-09,NaN,0


In [86]:
# Esta função não conta os NaN
df.count()

codigo_ocorrencia2          5752
ocorrencia_classificacao    5752
ocorrencia_cidade           5752
ocorrencia_uf               5752
ocorrencia_aerodromo        3553
ocorrencia_dia              5752
ocorrencia_hora             5751
total_recomendacoes         5752
dtype: int64

In [91]:
# Criando filtros

# Ocorrências com mais de 10 recomendações

filtrorec = df.total_recomendacoes > 10
df.loc[filtrorec, ['ocorrencia_cidade', 'total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
63,SÃO PAULO,11
326,RIO DE JANEIRO,19
530,SÃO PAULO,11
727,ITÁPOLIS,11
872,RECIFE,23
884,SÃO JOSÉ DOS PINHAIS,11
1062,UBERLÂNDIA,20
1972,BRASÍLIA,11
2795,SANTOS,13
3930,VITÓRIA,12


In [92]:
# Ocorrências graves

filtrograve = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtrograve, ['ocorrencia_cidade', 'total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
7,CANUTAMA,3
8,CASCAVEL,2
9,PARÁ DE MINAS,0
29,CURITIBA,2
35,PALMAS,0
...,...,...
5741,NOVA MARINGÁ,0
5744,GOIATUBA,0
5746,MANOEL URBANO,0
5748,LAGOA DA CONFUSÃO,0


In [95]:
Filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
Filtro2 = df.ocorrencia_uf == 'SP'
# filtro1 E filtro2
df.loc[Filtro1 & Filtro2]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
136,40270,INCIDENTE GRAVE,PIRASSUNUNGA,SP,SDPY,2010-04-15,20:15:00,0
171,40620,INCIDENTE GRAVE,GUARULHOS,SP,SBGR,2010-05-06,15:50:00,0
219,41411,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-06-08,19:30:00,2
297,42289,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-07-29,20:06:00,0
314,42323,INCIDENTE GRAVE,SÃO JOSÉ DO RIO PRETO,SP,SBSR,2010-08-05,12:01:00,0
...,...,...,...,...,...,...,...,...
5591,79540,INCIDENTE GRAVE,SÃO PAULO,SP,<NA>,2020-09-12,13:10:00,0
5610,79566,INCIDENTE GRAVE,REGENTE FEIJÓ,SP,SDYJ,2020-09-27,19:33:00,0
5704,79739,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2020-12-03,14:42:00,0
5705,79705,INCIDENTE GRAVE,SOROCABA,SP,SDCO,2020-12-04,15:30:00,0


In [96]:
#Filtro 1 OU Filtro2
df.loc[Filtro1 | Filtro2]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,INCIDENTE GRAVE,CANUTAMA,AM,<NA>,2010-01-09,12:30:00,3
8,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,<NA>,2010-01-10,20:00:00,0
11,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
16,39809,INCIDENTE,SÃO PAULO,SP,<NA>,2010-01-15,15:00:00,0
...,...,...,...,...,...,...,...,...
5744,79753,INCIDENTE GRAVE,GOIATUBA,GO,<NA>,2020-12-29,12:00:00,0
5746,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00,0
5747,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0


In [98]:
Filtro3 = (df.ocorrencia_classificacao == 'INCIDENTE GRAVE') | (
    df.ocorrencia_classificacao == 'INCIDENTE')
# tem de indicar as prioridades
Filtro2 = df.ocorrencia_uf == 'SP'
# filtro3 E filtro2
df.loc[Filtro3 & Filtro2]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
16,39809,INCIDENTE,SÃO PAULO,SP,<NA>,2010-01-15,15:00:00,0
18,39828,INCIDENTE,SANTOS,SP,<NA>,2010-01-15,17:45:00,0
26,39847,INCIDENTE,SOROCABA,SP,<NA>,2010-01-20,13:10:00,0
27,39768,INCIDENTE,CAMPINAS,SP,<NA>,2010-01-21,20:45:00,0
33,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
...,...,...,...,...,...,...,...,...
5704,79739,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2020-12-03,14:42:00,0
5705,79705,INCIDENTE GRAVE,SOROCABA,SP,SDCO,2020-12-04,15:30:00,0
5715,79718,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2020-12-11,13:50:00,0
5742,79800,INCIDENTE,SÃO PAULO,SP,SBMT,2020-12-28,10:15:00,0


In [100]:
filtroisin =  df.ocorrencia_classificacao.isin(['INCIDENTE GRAVE', 'INCIDENTE'])
df.loc[filtroisin & Filtro2]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
16,39809,INCIDENTE,SÃO PAULO,SP,<NA>,2010-01-15,15:00:00,0
18,39828,INCIDENTE,SANTOS,SP,<NA>,2010-01-15,17:45:00,0
26,39847,INCIDENTE,SOROCABA,SP,<NA>,2010-01-20,13:10:00,0
27,39768,INCIDENTE,CAMPINAS,SP,<NA>,2010-01-21,20:45:00,0
33,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
...,...,...,...,...,...,...,...,...
5704,79739,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2020-12-03,14:42:00,0
5705,79705,INCIDENTE GRAVE,SOROCABA,SP,SDCO,2020-12-04,15:30:00,0
5715,79718,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2020-12-11,13:50:00,0
5742,79800,INCIDENTE,SÃO PAULO,SP,SBMT,2020-12-28,10:15:00,0


In [102]:
# Analisando parte da valor da coluna:

# Todas as cidades que começam com a letra c

filtroc = df.ocorrencia_cidade.str[0] == 'C'
df.loc[filtroc]



,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
6,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,INCIDENTE GRAVE,CANUTAMA,AM,<NA>,2010-01-09,12:30:00,3
8,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
15,39315,ACIDENTE,CANUTAMA,AM,<NA>,2010-01-15,21:22:00,0
20,40310,INCIDENTE,CRUZEIRO DO SUL,AC,SBCZ,2010-01-18,19:32:00,0
...,...,...,...,...,...,...,...,...
5661,79649,INCIDENTE,CAMPINAS,SP,SBKP,2020-11-04,22:39:00,0
5663,79652,INCIDENTE,CONFINS,MG,SBCF,2020-11-06,14:20:00,0
5680,79683,INCIDENTE,CRUZEIRO DO SUL,AC,SBCZ,2020-11-16,22:00:00,0
5696,79692,ACIDENTE,CATANDUVA,SP,SDCD,2020-11-28,12:50:00,0


In [103]:
# Ocorrencias cujo as cidades terminam com a letra 'a'

filtroA = df.ocorrencia_cidade.str[-1] == 'A'
df.loc[filtroA]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,INCIDENTE GRAVE,CANUTAMA,AM,<NA>,2010-01-09,12:30:00,3
12,40414,INCIDENTE,BRASÍLIA,DF,SBBR,2010-01-10,03:00:00,0
15,39315,ACIDENTE,CANUTAMA,AM,<NA>,2010-01-15,21:22:00,0
25,39772,INCIDENTE,UBERLÂNDIA,MG,<NA>,2010-01-20,18:58:00,0
26,39847,INCIDENTE,SOROCABA,SP,<NA>,2010-01-20,13:10:00,0
...,...,...,...,...,...,...,...,...
5738,79787,INCIDENTE,FORTALEZA,CE,SBFZ,2020-12-23,18:30:00,0
5739,79749,ACIDENTE,MARITUBA,PA,<NA>,2020-12-26,12:05:00,0
5743,79824,ACIDENTE,RIO PARANAÍBA,MG,SNRP,2020-12-28,17:00:00,0
5744,79753,INCIDENTE GRAVE,GOIATUBA,GO,<NA>,2020-12-29,12:00:00,0


In [104]:
# todas as cidades que terminam com 'MA':

filtroma = df.ocorrencia_cidade.str[-2:] == 'MA'
df.loc[filtroma]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,INCIDENTE GRAVE,CANUTAMA,AM,<NA>,2010-01-09,12:30:00,3
15,39315,ACIDENTE,CANUTAMA,AM,<NA>,2010-01-15,21:22:00,0
408,43062,ACIDENTE,NOVO GAMA,GO,<NA>,2010-09-25,16:30:00,1
688,44198,ACIDENTE,BURITAMA,SP,<NA>,2011-03-19,11:18:00,0
1035,51785,INCIDENTE,BURITAMA,SP,SIBX,2011-10-25,19:15:00,0
1377,45939,ACIDENTE,UMUARAMA,PR,<NA>,2012-05-30,19:00:00,0
1401,46180,ACIDENTE,AURIFLAMA,SP,<NA>,2012-06-16,19:30:00,0
2099,49113,INCIDENTE GRAVE,NOVA LIMA,MG,<NA>,2013-06-15,12:00:00,4
2102,48799,ACIDENTE,CANUTAMA,AM,<NA>,2013-06-18,21:30:00,0
2655,51347,INCIDENTE,DIADEMA,SP,<NA>,2014-05-04,16:00:00,0


In [105]:
# Metodo contains - se contem 'MA' em qlq parte do texto.

filtrocontains = df.ocorrencia_cidade.str.contains('MA')
df.loc[filtrocontains]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,INCIDENTE GRAVE,CANUTAMA,AM,<NA>,2010-01-09,12:30:00,3
13,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,<NA>,2010-01-11,17:00:00,2
15,39315,ACIDENTE,CANUTAMA,AM,<NA>,2010-01-15,21:22:00,0
35,39487,INCIDENTE GRAVE,PALMAS,TO,SBPJ,2010-01-26,16:50:00,0
78,39916,INCIDENTE GRAVE,MANAUS,AM,SBEG,2010-03-07,19:35:00,2
...,...,...,...,...,...,...,...,...
5732,79745,INCIDENTE,MARABÁ,PA,SBMA,2020-12-21,08:47:00,0
5739,79749,ACIDENTE,MARITUBA,PA,<NA>,2020-12-26,12:05:00,0
5741,79754,INCIDENTE GRAVE,NOVA MARINGÁ,MT,<NA>,2020-12-28,13:00:00,0
5745,79755,ACIDENTE,MATO RICO,PR,<NA>,2020-12-29,10:30:00,0


In [106]:
# Metodo contains - se contem 'MA'  ou AL em qlq parte do texto.

filtromaal = df.ocorrencia_cidade.str.contains('MA | AL')
df.loc[filtromaal]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
110,40089,ACIDENTE,VILA BELA DA SANTÍSSIMA TRINDADE,MT,<NA>,2010-03-28,15:10:00,0
130,40258,INCIDENTE,MONTE ALEGRE,PA,<NA>,2010-04-12,18:10:00,0
131,40327,INCIDENTE,PORTO ALEGRE,RS,<NA>,2010-04-12,13:13:00,0
133,40248,INCIDENTE,PORTO ALEGRE,RS,SBPA,2010-04-13,18:47:00,0
156,40572,ACIDENTE,PORTO ALEGRE,RS,SBPA,2010-04-28,14:10:00,0
...,...,...,...,...,...,...,...,...
5652,79645,ACIDENTE,ALTO ALEGRE,RR,<NA>,2020-10-28,12:25:00,0
5659,79647,INCIDENTE,PORTO ALEGRE,RS,SBPA,2020-11-01,18:20:00,0
5679,79678,ACIDENTE,VILA BELA DA SANTÍSSIMA TRINDADE,MT,<NA>,2020-11-16,14:15:00,0
5682,79684,INCIDENTE,PORTO ALEGRE,RS,SBPA,2020-11-17,20:18:00,0


In [108]:
# Filtros com datas:

#precisamos opter ocorrencias de 2015

filtro2015 = df.ocorrencia_dia.dt.year == 2015
df.loc[filtro2015]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2996,52992,INCIDENTE GRAVE,SALVADOR,BA,SBSV,2015-01-01,11:40:00,3
2997,52979,ACIDENTE,IVINHEMA,MS,<NA>,2015-01-02,14:50:00,0
2998,53073,INCIDENTE,TEFÉ,AM,SBTF,2015-01-02,16:28:00,0
2999,53074,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-01-02,20:34:00,0
3000,52976,ACIDENTE,TOLEDO,PR,SBTD,2015-01-04,22:04:00,6
...,...,...,...,...,...,...,...,...
3462,60632,INCIDENTE GRAVE,ITABERÁ,SP,<NA>,2015-12-24,14:00:00,0
3463,60600,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-25,19:00:00,0
3464,60642,INCIDENTE,SÃO FRANCISCO DO SUL,SC,SSSS,2015-12-26,16:00:00,0
3465,60631,ACIDENTE,MAÇAMBARÁ,RS,<NA>,2015-12-28,19:00:00,2


In [109]:
# precisamos opter ocorrencias de 2015 e do mês 12

filtro2015 = df.ocorrencia_dia.dt.year == 2015
filtro12 = df.ocorrencia_dia.dt.month == 12
df.loc[filtro2015 & filtro12]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3428,53573,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-01,02:48:00,0
3429,60601,INCIDENTE,PALMAS,TO,SBPJ,2015-12-01,16:05:00,0
3430,53634,INCIDENTE,PALMAS,TO,SBPJ,2015-12-02,17:45:00,0
3431,53636,INCIDENTE,JUNDIAÍ,SP,SBJD,2015-12-02,17:42:00,0
3432,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3433,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3434,53625,ACIDENTE,TRINDADE,GO,<NA>,2015-12-06,13:10:00,3
3435,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3436,53628,ACIDENTE,AGUAÍ,SP,<NA>,2015-12-08,14:30:00,1
3437,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0


In [110]:
# precisamos opter ocorrencias de 2015 e do mês 12

filtro122015 = (df.ocorrencia_dia.dt.year == 2015) & (
    df.ocorrencia_dia.dt.month == 12)
df.loc[filtro122015]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3428,53573,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-01,02:48:00,0
3429,60601,INCIDENTE,PALMAS,TO,SBPJ,2015-12-01,16:05:00,0
3430,53634,INCIDENTE,PALMAS,TO,SBPJ,2015-12-02,17:45:00,0
3431,53636,INCIDENTE,JUNDIAÍ,SP,SBJD,2015-12-02,17:42:00,0
3432,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3433,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3434,53625,ACIDENTE,TRINDADE,GO,<NA>,2015-12-06,13:10:00,3
3435,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3436,53628,ACIDENTE,AGUAÍ,SP,<NA>,2015-12-08,14:30:00,1
3437,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0


In [111]:
# dentro dia 8 de 12 de 2015

filtro08122015 = (df.ocorrencia_dia.dt.year == 2015) &
( df.ocorrencia_dia.dt.month == 12) & ( df.ocorrencia_dia.dt.day == 08)
df.loc[filtro08122005]

SyntaxError: invalid syntax (<ipython-input-111-68cb2bc9be59>, line 3)

In [114]:
filtro2015 = df.ocorrencia_dia.dt.year == 2015
filtro12 = df.ocorrencia_dia.dt.month == 12
filtro08 = df.ocorrencia_dia.dt.day == 8
df.loc[filtro2015 & filtro12 & filtro08]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3436,53628,ACIDENTE,AGUAÍ,SP,<NA>,2015-12-08,14:30:00,1
3437,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0
3438,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0
3439,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0


In [117]:
# filtrando do dia 3 ao dia 8:
filtro_dia = (df.ocorrencia_dia.dt.day > 2) & (df.ocorrencia_dia.dt.day < 9)
df.loc[filtro2015 & filtro12 & filtro_dia]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3432,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3433,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3434,53625,ACIDENTE,TRINDADE,GO,<NA>,2015-12-06,13:10:00,3
3435,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3436,53628,ACIDENTE,AGUAÍ,SP,<NA>,2015-12-08,14:30:00,1
3437,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0
3438,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0
3439,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0


In [119]:
#filtrar a hora


#primeiro transofrmar a ocorrencia_data como str


#criando nova coluna
df['Ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora )

In [120]:
df.head()

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,Ocorrencia_dia_hora
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0,2010-01-03 12:00:00
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,2010-01-03 11:05:00
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0,2010-01-04 17:30:00
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0,2010-01-05 19:25:00


In [121]:
df.dtypes

codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
Ocorrencia_dia_hora         datetime64[ns]
dtype: object

In [125]:
filtro_ano = df.Ocorrencia_dia_hora.dt.year == 2015
filtro_mes= df.Ocorrencia_dia_hora.dt.month == 12
filtro_dia = (df.Ocorrencia_dia_hora.dt.day > 2) & (df.Ocorrencia_dia_hora.dt.day < 9)

df.loc[filtro_ano & filtro_mes & filtro_dia ]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,Ocorrencia_dia_hora
3432,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0,2015-12-03 10:50:00
3433,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0,2015-12-03 16:47:00
3434,53625,ACIDENTE,TRINDADE,GO,<NA>,2015-12-06,13:10:00,3,2015-12-06 13:10:00
3435,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1,2015-12-06 15:00:00
3436,53628,ACIDENTE,AGUAÍ,SP,<NA>,2015-12-08,14:30:00,1,2015-12-08 14:30:00
3437,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0,2015-12-08 10:20:00
3438,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0,2015-12-08 16:19:00
3439,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0,2015-12-08 13:00:00


In [127]:
#fltro um periodo de horas
filtroh1 = df.Ocorrencia_dia_hora >= '2015-12-03 11:00:00'
filtroh2 = df.Ocorrencia_dia_hora <= '2015-12-08 13:00:00'
df.loc[filtroh1 & filtroh2]

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,Ocorrencia_dia_hora
3433,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0,2015-12-03 16:47:00
3434,53625,ACIDENTE,TRINDADE,GO,<NA>,2015-12-06,13:10:00,3,2015-12-06 13:10:00
3435,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1,2015-12-06 15:00:00
3437,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0,2015-12-08 10:20:00
3439,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0,2015-12-08 13:00:00


In [128]:
# criando um novo dataframe com  o ano 2015 e mês 3
filtro_ano = df.ocorrencia_dia.dt.year == 2015
filtro_mes = df.ocorrencia_dia.dt.month == 3
df201503 =df.loc[filtro_ano & filtro_mes]

In [131]:
df201503.fillna(np.nan)

,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,Ocorrencia_dia_hora
3086,53183,ACIDENTE,AMAPORÃ,PR,NaN,2015-03-02,23:00:00,0,2015-03-02 23:00:00
3087,53120,ACIDENTE,CHAVES,PA,SNXW,2015-03-04,13:30:00,0,2015-03-04 13:30:00
3088,53109,ACIDENTE,CAMPO GRANDE,MS,SSIE,2015-03-05,13:50:00,0,2015-03-05 13:50:00
3089,53112,ACIDENTE,MOGI GUAÇU,SP,NaN,2015-03-06,21:00:00,0,2015-03-06 21:00:00
3090,53152,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBRJ,2015-03-10,11:30:00,2,2015-03-10 11:30:00
3091,53167,INCIDENTE,MARABÁ,PA,SBMA,2015-03-10,17:33:00,0,2015-03-10 17:33:00
3092,53596,INCIDENTE,ITAPEMA,SC,NaN,2015-03-10,21:35:00,0,2015-03-10 21:35:00
3093,53149,ACIDENTE,TABATINGA,AM,SBTT,2015-03-11,23:25:00,0,2015-03-11 23:25:00
3094,53148,ACIDENTE,ARARAS,SP,SDEH,2015-03-12,12:40:00,0,2015-03-12 12:40:00
3095,53153,INCIDENTE,PORTO ALEGRE,RS,SBPA,2015-03-13,15:45:00,0,2015-03-13 15:45:00


In [134]:
df201503.count()

codigo_ocorrencia2          37
ocorrencia_classificacao    37
ocorrencia_cidade           37
ocorrencia_uf               37
ocorrencia_aerodromo        21
ocorrencia_dia              37
ocorrencia_hora             37
total_recomendacoes         37
Ocorrencia_dia_hora         37
dtype: int64

In [137]:
#Agrupando
df201503.groupby(['codigo_ocorrencia2']).count()
# 1 existe 
# 0 não existe

,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,Ocorrencia_dia_hora
codigo_ocorrencia2,,,,,,,,
53109,1,1,1,1,1,1,1,1
53112,1,1,1,0,1,1,1,1
53120,1,1,1,1,1,1,1,1
53148,1,1,1,1,1,1,1,1
53149,1,1,1,1,1,1,1,1
53151,1,1,1,1,1,1,1,1
53152,1,1,1,1,1,1,1,1
53153,1,1,1,1,1,1,1,1
53154,1,1,1,1,1,1,1,1
